<a href="https://colab.research.google.com/github/directin/LLM_Gemini/blob/main/Alura_aula5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install -u -q google-gnetativeai



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [4]:
!pip install -U -q google-generativeai


In [45]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [46]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


In [47]:
#Exemplo de texto
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo completo:\n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessivel de explorar e criar protótipos com aplicações de IA generativa")
embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.0512451, -0.036198843, -0.019158708, 0.019573186, 0.06742889, -0.0052685305, -0.02767586, -0.018914977, 0.06220338, 0.06257593, 0.011453855, 0.013132725, -0.046880096, -0.021972213, 0.009486085, -0.026625467, 0.022598978, -0.012096425, -0.034788493, -0.0011314827, 0.007497891, 0.0059401016, -0.029543951, -0.06606445, -0.020090975, 0.02098767, 0.015545649, -0.037946403, -0.03326899, 0.02681229, -0.049690086, 0.049942255, -0.033831, 0.010327337, -0.035956822, -0.040874034, -0.026168479, -0.04471691, -0.0021545717, -0.0033930347, 0.0056297192, -0.08592187, -0.010266354, 0.026619304, -0.0033729204, -0.02107747, 0.044365887, 0.04377648, 0.014638021, -0.05903468, 0.026893962, 0.022291362, 0.06483139, -0.04236089, -0.0048635313, -0.009288106, 0.023343146, -0.033797406, 0.024214797, -0.0003407756, -0.0009546803, 0.023094537, -0.016091159, 0.05340161, 0.018512271, -0.0583988, -0.04219819, -1.6810434e-05, 0.011712833, 0.048436727, 0.0098303985, 0.013133031, 0.061246675, -0.06364

In [48]:
DOCUMENT1 ={
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema"}

DOCUMENT2 ={
    "Título": "Touchscreen",
    "Conteúdo": "Seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação"}
DOCUMENT3 ={
    "Título": "Mudança de marchar",
    "Conteúdo": "O Googlecar tem uma transmissão automática. Para trocar as marchas basta mover a alavanca de câmbio para posição desejada"}

documents = [DOCUMENT1,DOCUMENT2,DOCUMENT3]

In [49]:
df = pd.DataFrame(documents)
df.columns = ["Titulo",'Conteudo']
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,Seu Googlecar tem uma grande tela sensível ao ...
2,Mudança de marchar,O Googlecar tem uma transmissão automática. Pa...


In [50]:
model = "models/embedding-001"


In [51]:
def embed_fn(title, text):
   return genai.embed_content(model=model,
                              content=text,
                              title=title,
                              task_type="RETRIEVAL_DOCUMENT")["embedding"]


In [52]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]),axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.010924981, -0.016569521, -0.055676658, 0.0..."
1,Touchscreen,Seu Googlecar tem uma grande tela sensível ao ...,"[0.011963523, -0.051407754, -0.02546281, -0.02..."
2,Mudança de marchar,O Googlecar tem uma transmissão automática. Pa...,"[0.0017612239, -0.037485052, -0.008915988, 0.0..."


In [53]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                              content=consulta,
                              task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"], embedding_da_consulta))
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [54]:
consulta = "Como faço para trocar marchas em carro do Google ?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

TypeError: 'list' object cannot be interpreted as an integer